Рассматриваются связные неориентированные взвешенные (все веса - положительные числа) графы без кратных ребер и петель. Требуется найти любой простой путь заданной длины между двумя заданными вершинами.

Описание алгоритма Йена (модифицированный алгоритм Дейкстры; изначально задумывался как алгоритм поиска альтернативных путей между заданными вершинами, но легко можно переделать для поиска простого пути заданной длины):

1) Применяется алгоритм Дейкстры, запоминается путь и длина. Путь хранится в списке paths, вес - в списке weight_of_paths.
2) Рассматривается последний элемент (путь) списка paths.
   Удаляется первое ребро. Запускается алгоритм Дейкстры (если конечная вершина осталась достижима из начальной). Найденный путь сохраняется в списке candidates, его вес - в списке weught_of_candidates.
   Второй найденный путь будет состоять из первого ребра последнего пути из paths и нового подпути, который будет получен следующим образом: удаляется первая вершина рассматриваемого пути из paths и все инцидентные ей ребра. Также удаляется второе ребро рассматриваемого пути из paths. Вторая вершина пути из paths становится вершиной ветвления - елси конечная вершина осталась достижима из нее, то запускается Дейкстра от вершины ветвления до конечной. Найденный путь добавляется в candidates, его вес - в weight_of_candidates. После обработки последенего пути из paths в paths добавляется в конец путь с наименьшим весом из candidates, его вес добавляется в weight_of_paths.
3) Как только в candidates добавлен путь с искомой длиной алгоритм останавливается и на экран выводится найденный путь и его длина. Если путь найден при первом запуске Дейкстры (candidates пуст), то возвращается путь, найденный Дейкстрой (кратчайший).

При тестировании сначала ищется 10-тысячный путь в графе между начальной и конечной вершиной, а потом еще раз запускается алгоритм Йена и ищется первый путь с такой длиной (при работе алгоритма Йена веса путей попадаются в произвольном порядке, поэтому такой вес мог быть и раньше).

Описание моего алгоритма:

1) Рассматриваются все ребра между начальной и конечной вершиной. Если найдено ребро, вес которого равен весу искомого пути, то алгоритм останавливается и возвращается найденное ребро (зачем это делается станет понятно позже).
2) Начальная и конечная вершина при дальнейших преобразования не рассматриваются. Находится наименьшая степень вершины. Расматриваются поочередно все вершины, степень которых равна наименьшей. Вершина удаляется из графа. Инцидентные ей ребра "склеиваются" в новые следующим образом (вес нового ребра равен сумме весов склееных):
   1. В каждом ребер хранится информация о том, через какие вершины в начальном графе оно проходило. Склеиваются только те ребра, у которых множества промежуточных вершин не пересекаются. В новом ребре список с промежуточными вершинами равен списку первого ребра + вершина склейки + список вотрого ребра (иногда нужно делать reverse одного или двух списков, чтобы информация правильно хранилась).
   2. В графе в общем случае возникают кратные ребра. Не склеиваются ребра, которые при склейке образовали бы петлю, т.е. [v, u, w_1, list_1] и [v, u, w_2, list_2] не склеиваются, т.к. образуется петля - вершина u проходится 2 раза (если v - вершина склейки).
   3. До проведения склеек находится наименьший вес ребра. Возможны 3 случая:
      - При склейке образуется ребро между начальной и конечной вершиной. Оно проводится, если и только если его вес равен весу искомого пути. Если такое ребро провелось, то алгоритм останавливается, из этого ребра извлекается путь в исходном графе и выводистя на экран вместе с весом.
      - При склейке образуется ребро между начальной или конечной вершиной и любой промежуточной. Тогда ребро проводится только в том случае, если его вес + минимальный вес <= весу искомого пути (потребуется хотя бы одно ребро, чтобы дойти до конечной или начальной).
      - При склейке образуется ребро между двумя промежуточными вершинами. Тогда ребро проводится только в том случае, елси его вес + 2*минимальный вес <= весу искомого пути (потребуется хотя бы 2 ребра, чтобы соединить начальную и конечную вершину через данное ребро).
   Это продолжается до тех пор, пока не будет проведено первое ребро между начальной и конечной вершиной. Если в графе единственный путь искомой длины между заданными вершинами представляет собой ребро между ними, то его проще найти сразу, проверив все ребра, что мы и сделали в 1).

In [ ]:
from networkx.generators.random_graphs import erdos_renyi_graph
import networkx as nx
from random import randint
import time

n = 30 #количество вершин в графе
p = 0.25 #вероятность проведения ребра

STATIONS = {}
for i in range(n):
    STATIONS[i + 1] = i + 1

LINKS = []

g_1 = erdos_renyi_graph(n, p) 
while not nx.is_connected(g_1):
    g_1 = erdos_renyi_graph(n, p)

print(len(g_1.nodes), len(g_1.edges))

g = nx.Graph()

edges_with_weights = []

for i in range(n):
    g.add_node(i)
    adj_list = g_1.adj[i]
    for u in adj_list:
        if u > i:
            w = randint(1, 100)
            g.add_edge(i, u, weight = w)
            edges_with_weights.append([i, u, w])
            LINKS.append([i + 1, u + 1, w])

paths = []
weight_of_paths = []
candidates = []
weight_of_candidates = []

start_node = 4  #дальше на 1 больше, т.к. свертка работает с нумерацией с 1
end_node = 15

cnt = 10**4 #количество просатриваемых путей алгоритмом Йена в первый раз

#первый запуск алгоритма Йена
count_paths = 0
paths.append(nx.dijkstra_path(g, start_node, end_node))
weight_of_paths.append(nx.dijkstra_path_length(g, start_node, end_node))
count_paths += 1

for i in range(len(paths[0]) - 1):
    if i == 0:
        w = g.edges[paths[0][0], paths[0][1]]['weight']
        g.remove_edge(paths[0][0], paths[0][1])
        if end_node in nx.algorithms.descendants(g, paths[0][i]) and not nx.dijkstra_path(g, start_node, end_node) in candidates:
            candidates.append(nx.dijkstra_path(g, start_node, end_node))
            weight_of_candidates.append(nx.dijkstra_path_length(g, start_node, end_node))
            count_paths += 1
        g.add_edge(paths[0][0], paths[0][1], weight = w)
        if count_paths == cnt:
            weight_of_path = weight_of_candidates[-1]
            print('path', candidates[-1])
            print('weight of path', weight_of_candidates[-1])
            break

    else:
        w = g.edges[paths[0][i], paths[0][i + 1]]['weight']
        w_of_way = 0
        way = []
        for j in range(i):
            w_of_way += g.edges[paths[0][j], paths[0][j + 1]]['weight']
            g.remove_node(paths[0][j])
            way.append(paths[0][j])
        g.remove_edge(paths[0][i], paths[0][i + 1])
        if end_node in nx.algorithms.descendants(g, paths[0][i]): 
            end_way = nx.dijkstra_path(g, paths[0][i], end_node)
            if not (way + end_way) in candidates:
                candidates.append(way + end_way)
                weight_of_candidates.append(w_of_way + nx.dijkstra_path_length(g, paths[0][i], end_node))
                count_paths += 1
        for j in range(len(way)):
            g.add_node(way[j])
        for v in way:
            for j in range(len(edges_with_weights)):
                if edges_with_weights[j][0] == v:
                    g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
                elif edges_with_weights[j][1] == v:
                    g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
        g.add_edge(paths[0][i], paths[0][i + 1], weight = w)

    if count_paths == cnt:
        weight_of_path = weight_of_candidates[-1]
        print('path', candidates[-1])
        print('weight of path', weight_of_candidates[-1])
        break

min_ind = weight_of_candidates.index(min(weight_of_candidates))
paths.append(candidates[min_ind])
weight_of_paths.append(weight_of_candidates[min_ind])
candidates.pop(min_ind) #надо удалить подходящий путь, иначе он будет все время добавлять его
weight_of_candidates.pop(min_ind)

n = 0

while count_paths < cnt:
    n += 1 #номер пути из paths
    for i in range(len(paths[n])):
        if i == 0:
            del_node = [] #список удаленных концов ребер для восстановления
            for j in range(len(paths)):
                if (paths[j][0], paths[j][1]) in g.edges:
                    g.remove_edge(paths[j][0], paths[j][1])
                    del_node.append(paths[j][1])
            if end_node in nx.algorithms.descendants(g, paths[n][i]) and not nx.dijkstra_path(g, start_node, end_node) in candidates:
                candidates.append(nx.dijkstra_path(g, start_node, end_node))
                weight_of_candidates.append(nx.dijkstra_path_length(g, start_node, end_node))
                count_paths += 1
                if count_paths == cnt:
                    weight_of_path = weight_of_candidates[-1]
                    print('path', candidates[-1])
                    print('weight of path', weight_of_candidates[-1])
            for v in del_node:
                for j in range(len(edges_with_weights)):
                    if edges_with_weights[j][0] == start_node and edges_with_weights[j][1] == v or edges_with_weights[j][0] == v and edges_with_weights[j][1] == start_node:
                        g.add_edge(start_node, v, weight = edges_with_weights[j][2])
        elif i > 0 and paths[n][i] != end_node:
            w_of_way = 0
            way = []
            for j in range(i):
                way.append(paths[n][j])
                w_of_way += g.edges[paths[n][j], paths[n][j + 1]]['weight']
                g.remove_node(paths[n][j])
            way.append(paths[n][i])
            adj_list = g.adj[paths[n][i]]
            k = 0
            for j in range(len(paths)):
                if len(paths[j]) >= len(way) + 1:  #+1 из-за того, что потом добавим вершину
                    k += 1
            b = 0 #проверка на добавление нового пути в candidates
            for v in adj_list:
                if v != paths[n][i - 1]:
                    way.append(v)
                    k_check = 0
                    for j in range(len(paths)):
                        if len(paths[j]) >= len(way) and paths[j][0:len(way):1] != way: #такого начала нигде не должно быть, а не только в одном пути
                            k_check += 1
                    if k_check < k:
                        way.pop(-1)
                    else:
                        way.pop(-1)
                        w_of_way += g.edges[paths[n][i], v]['weight']
                        g.remove_node(paths[n][i])
                        if end_node in nx.algorithms.descendants(g, v): #проверка, что v и end_node в одной комоненте связности                       
                            end_way = nx.dijkstra_path(g, v, end_node) 
                            if not (way + end_way) in candidates: 
                                candidates.append(way + end_way)
                                weight_of_candidates.append(w_of_way + nx.dijkstra_path_length(g, v, end_node))
                                count_paths += 1
                                if count_paths == cnt:
                                    weight_of_path = weight_of_candidates[-1]
                                    print('path', candidates[-1])
                                    print('weight of path', weight_of_candidates[-1])
                                    print()
                                b = 1
                        g.add_node(paths[n][i])   #вернуть удаленную вершину, ребра и исправить вес нового пути
                        for a in range(len(edges_with_weights)):
                            if (edges_with_weights[a][0] == paths[n][i] and edges_with_weights[a][1] in g.nodes) or (edges_with_weights[a][1] == paths[n][i] and edges_with_weights[a][0] in g.nodes):
                                g.add_edge(edges_with_weights[a][0], edges_with_weights[a][1], weight = edges_with_weights[a][2])
                        w_of_way -= g.edges[paths[n][i], v]['weight']
                        if b == 1:
                            break
            for j in range(len(way) - 1):
                g.add_node(way[j])
            for v in way:
                for j in range(len(edges_with_weights)):
                    if edges_with_weights[j][0] == v:
                        g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
                    elif edges_with_weights[j][1] == v:
                        g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
    if len(weight_of_candidates) > 0:
        min_ind = weight_of_candidates.index(min(weight_of_candidates))
        paths.append(candidates[min_ind])
        weight_of_paths.append(weight_of_candidates[min_ind])
        candidates.pop(min_ind)
        weight_of_candidates.pop(min_ind)
    else:
        break

#второй запуск алгоритма Йена
start_time = time.time()

while True:
    paths = []
    weight_of_paths = []
    candidates = []
    weight_of_candidates = []

    count_paths = 0
    paths.append(nx.dijkstra_path(g, start_node, end_node))
    weight_of_paths.append(nx.dijkstra_path_length(g, start_node, end_node))
    count_paths += 1

    if weight_of_paths[-1] == weight_of_path:
        print('path', paths[0])
        print('weight path', weight_of_paths[0])
        print('count paths', count_paths)
        print()
        break

    for i in range(len(paths[0]) - 1):
        if i == 0:
            w = g.edges[paths[0][0], paths[0][1]]['weight']
            g.remove_edge(paths[0][0], paths[0][1])
            if end_node in nx.algorithms.descendants(g, paths[0][i]) and not nx.dijkstra_path(g, start_node, end_node) in candidates:
                candidates.append(nx.dijkstra_path(g, start_node, end_node))
                weight_of_candidates.append(nx.dijkstra_path_length(g, start_node, end_node))
                count_paths += 1
                if weight_of_candidates[-1] == weight_of_path:
                    print('path', candidates[-1])
                    print('weight path', weight_of_candidates[-1])
                    print('count paths', count_paths)
                    print()
                    break
            g.add_edge(paths[0][0], paths[0][1], weight = w)

        else:
            w = g.edges[paths[0][i], paths[0][i + 1]]['weight']
            w_of_way = 0
            way = []
            for j in range(i):
                w_of_way += g.edges[paths[0][j], paths[0][j + 1]]['weight']
                g.remove_node(paths[0][j])
                way.append(paths[0][j])
            g.remove_edge(paths[0][i], paths[0][i + 1])
            if end_node in nx.algorithms.descendants(g, paths[0][i]): 
                end_way = nx.dijkstra_path(g, paths[0][i], end_node)
                if not (way + end_way) in candidates:
                    candidates.append(way + end_way)
                    weight_of_candidates.append(w_of_way + nx.dijkstra_path_length(g, paths[0][i], end_node))
                    count_paths += 1
            for j in range(len(way)):
                g.add_node(way[j])
            for v in way:
                for j in range(len(edges_with_weights)):
                    if edges_with_weights[j][0] == v:
                        g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
                    elif edges_with_weights[j][1] == v:
                        g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
            g.add_edge(paths[0][i], paths[0][i + 1], weight = w)
            if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
                print('path', candidates[-1])
                print('weight path', weight_of_candidates[-1])
                print('count paths', count_paths)
                print()
                break

    if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
        break

    min_ind = weight_of_candidates.index(min(weight_of_candidates))
    paths.append(candidates[min_ind])
    weight_of_paths.append(weight_of_candidates[min_ind])
    candidates.pop(min_ind) #надо удалить подходящий путь, иначе он будет все время добавлять его
    weight_of_candidates.pop(min_ind)

    n = 0

    while count_paths < cnt:
        n += 1 #номер пути из paths
        for i in range(len(paths[n])):
            if i == 0:
                del_node = [] #список удаленных концов ребер для восстановления
                for j in range(len(paths)):
                    if (paths[j][0], paths[j][1]) in g.edges:
                        g.remove_edge(paths[j][0], paths[j][1])
                        del_node.append(paths[j][1])
                if end_node in nx.algorithms.descendants(g, paths[n][i]) and not nx.dijkstra_path(g, start_node, end_node) in candidates:
                    candidates.append(nx.dijkstra_path(g, start_node, end_node))
                    weight_of_candidates.append(nx.dijkstra_path_length(g, start_node, end_node))
                    count_paths += 1
                    if weight_of_candidates[-1] == weight_of_path:
                        print('path', candidates[-1])
                        print('weight path', weight_of_candidates[-1])
                        print('count pahs', count_paths)
                        print()
                        break
                for v in del_node:
                    for j in range(len(edges_with_weights)):
                        if edges_with_weights[j][0] == start_node and edges_with_weights[j][1] == v or edges_with_weights[j][0] == v and edges_with_weights[j][1] == start_node:
                            g.add_edge(start_node, v, weight = edges_with_weights[j][2])
                if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
                    break
            elif len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
                break
            elif i > 0 and paths[n][i] != end_node:
                w_of_way = 0
                way = []
                for j in range(i):
                    way.append(paths[n][j])
                    w_of_way += g.edges[paths[n][j], paths[n][j + 1]]['weight']
                    g.remove_node(paths[n][j])
                way.append(paths[n][i])
                adj_list = g.adj[paths[n][i]]
                k = 0
                for j in range(len(paths)):
                    if len(paths[j]) >= len(way) + 1:  #+1 из-за того, что потом добавим вершину
                        k += 1
                b = 0 #проверка на добавление нового пути в candidates
                for v in adj_list:
                    if v != paths[n][i - 1]:
                        way.append(v)
                        k_check = 0
                        for j in range(len(paths)):
                            if len(paths[j]) >= len(way) and paths[j][0:len(way):1] != way: #такого начала нигде не должно быть, а не только в одном пути
                                k_check += 1
                        if k_check < k:
                            way.pop(-1)
                        else:
                            way.pop(-1)
                            w_of_way += g.edges[paths[n][i], v]['weight']
                            g.remove_node(paths[n][i])
                            if end_node in nx.algorithms.descendants(g, v): #проверка, что v и end_node в одной комоненте связности                       
                                end_way = nx.dijkstra_path(g, v, end_node) 
                                if not (way + end_way) in candidates: 
                                    candidates.append(way + end_way)
                                    weight_of_candidates.append(w_of_way + nx.dijkstra_path_length(g, v, end_node))
                                    count_paths += 1
                                    b = 1
                                    if weight_of_candidates[-1] == weight_of_path:
                                        print('path', candidates[-1])
                                        print('weight path', weight_of_candidates[-1])
                                        print('count paths', count_paths)
                                        print()
                                        break
                            g.add_node(paths[n][i])   #вернуть удаленную вершину, ребра и исправить вес нового пути
                            for a in range(len(edges_with_weights)):
                                if (edges_with_weights[a][0] == paths[n][i] and edges_with_weights[a][1] in g.nodes) or (edges_with_weights[a][1] == paths[n][i] and edges_with_weights[a][0] in g.nodes):
                                    g.add_edge(edges_with_weights[a][0], edges_with_weights[a][1], weight = edges_with_weights[a][2])
                            w_of_way -= g.edges[paths[n][i], v]['weight']
                            if b == 1:
                                break
                        if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
                            break
                for j in range(len(way) - 1):
                    g.add_node(way[j])
                for v in way:
                    for j in range(len(edges_with_weights)):
                        if edges_with_weights[j][0] == v:
                            g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
                        elif edges_with_weights[j][1] == v:
                            g.add_edge(edges_with_weights[j][0], edges_with_weights[j][1], weight = edges_with_weights[j][2])
                if weight_of_candidates[-1] == weight_of_path:
                    break
        if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
            break
        if len(weight_of_candidates) > 0:
            min_ind = weight_of_candidates.index(min(weight_of_candidates))
            paths.append(candidates[min_ind])
            weight_of_paths.append(weight_of_candidates[min_ind])
            candidates.pop(min_ind)
            weight_of_candidates.pop(min_ind)
        else:
            break
    
    if len(weight_of_candidates) != 0 and weight_of_candidates[-1] == weight_of_path:
        break

end_time = time.time()
print('Алгоритм Йена', end_time - start_time)
print()

#запуск моего алгоритма
start_time = time.time()

#функция поиска наименьшего веса
def find_min_weight(list_of_links):
    m_w = 10000000
    for i in range(len(list_of_links)):
        if list_of_links[i][2] < m_w:
            m_w = list_of_links[i][2]
    return m_w

#функция "переворачивания" списка
def rev_list(v_list):
    result = []
    for i in range(len(v_list)-1, -1, -1):
        result.append(v_list[i])
    return result

start_node += 1
end_node += 1

LINKS_copy = []
first_check = False

#проверка на наличие ребра между нач и кон, вес которого равен весу искомого пути
for i in range(len(LINKS)):
    if ((LINKS[i][0] == start_node and LINKS[i][1] == end_node) or (LINKS[i][0] == end_node and LINKS[i][1] == start_node)) and LINKS[i][2] == weight_of_path:
        LINKS_copy.append([start_node, end_node, LINKS[i][2], []])
        first_check = True
        break

if not first_check:    

    deg_nodes = []     #список степеней вершин

    for i in range(len(STATIONS)+1):
        deg_nodes.append(0)

    for i in range(len(LINKS)):
        deg_nodes[LINKS[i][0]-1] += 1
        deg_nodes[LINKS[i][1]-1] += 1

    adjacency_list = {}       #список смежности вершин
    for i in range(1, len(STATIONS)+1):
        adjacency_list[i] = []

    for i in LINKS:
        adjacency_list[i[0]].append(i[1])
        adjacency_list[i[1]].append(i[0])

    STATIONS_copy = {}           #копия словаря вершин для преобразований
    for i in range(1, len(STATIONS)+1):
        STATIONS_copy[i] = STATIONS[i]

    LINKS_copy = []             #копия списка ребер для преобразований
    for i in range(len(LINKS)):
        LINKS_copy.append(list(LINKS[i]))
        LINKS_copy[i].append(list())
        
    del_deg = 10000000

    for j in range(len(deg_nodes)):
        if j+1 != start_node and j+1 != end_node and deg_nodes[j] < del_deg and deg_nodes[j] != 0:
            del_deg = deg_nodes[j]

    check = True
    while check:
        while (del_deg in deg_nodes and deg_nodes[start_node -1] != del_deg and deg_nodes[end_node -1] != del_deg) or (deg_nodes.count(del_deg) > 1 and (deg_nodes[start_node -1] == del_deg or deg_nodes[end_node -1] == del_deg) and deg_nodes[start_node -1] != deg_nodes[end_node -1]) or (deg_nodes.count(del_deg) > 2 and deg_nodes[start_node -1] == del_deg and deg_nodes[end_node -1] == del_deg):
            for i in range(len(deg_nodes)):
                if deg_nodes.count(del_deg) != 0 and deg_nodes[i] == del_deg and i+1 != start_node and i+1 != end_node:

                    del STATIONS_copy[i+1]

                    if del_deg == 1:
                        for j in range(len(LINKS_copy)):
                            if LINKS_copy[j][0] == i+1 or LINKS_copy[j][1] == i+1:
                                LINKS_copy.pop(j)
                                break
                    else:
                        new_weight = 0
                        min_weight = find_min_weight(LINKS_copy)
                        numbers_del = []
                        len_1 = len(LINKS_copy)
                        for j in range(len_1): 
                            if LINKS_copy[j][0] == i+1:
                                new_weight = LINKS_copy[j][2]
                                numbers_del.append(j)
                                for a in range(j+1, len_1): #ниже второе - проверка на то, что не проводим петлю
                                    if ((LINKS_copy[a][0] == i+1 and LINKS_copy[a][1] != LINKS_copy[j][1]) or (LINKS_copy[a][1] == i+1 and LINKS_copy[a][0] != LINKS_copy[j][1])) and (set(LINKS_copy[a][3]).intersection(set(LINKS_copy[j][3])) == set() or (len(LINKS_copy[a][3]) == 0 and len(LINKS_copy[j][3]) == 0)):
                                        new_weight += LINKS_copy[a][2]
                                        if LINKS_copy[a][0] == i+1 and (new_weight == weight_of_path or new_weight + min_weight <= weight_of_path or new_weight + 2 * min_weight <= weight_of_path):
                                            if new_weight == weight_of_path and ((LINKS_copy[j][1] == start_node and LINKS_copy[a][1] == end_node) or (LINKS_copy[j][1] == end_node and LINKS_copy[a][1] == start_node)):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][1], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                                check = False
                                                break
                                            elif new_weight + min_weight <= weight_of_path and ((LINKS_copy[j][1] == start_node and LINKS_copy[a][1] != end_node) or (LINKS_copy[j][1] == end_node and LINKS_copy[a][1] != start_node) or (LINKS_copy[j][1] != start_node and LINKS_copy[a][1] == end_node) or (LINKS_copy[j][1] != end_node and LINKS_copy[a][1] == start_node)):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][1], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                            elif new_weight + 2 * min_weight <= weight_of_path and (LINKS_copy[j][1] != start_node and LINKS_copy[a][1] != end_node) and (LINKS_copy[j][1] != end_node and LINKS_copy[a][1] != start_node):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][1], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                        elif LINKS_copy[a][1] == i+1 and (new_weight == weight_of_path or new_weight + min_weight <= weight_of_path or new_weight + 2 * min_weight <= weight_of_path):
                                            if new_weight == weight_of_path and ((LINKS_copy[j][1] == start_node and LINKS_copy[a][0] == end_node) or (LINKS_copy[j][1] == end_node and LINKS_copy[a][0] == start_node)):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                                check = False
                                                break
                                            elif new_weight + min_weight <= weight_of_path and ((LINKS_copy[j][1] == start_node and LINKS_copy[a][0] != end_node) or (LINKS_copy[j][1] == end_node and LINKS_copy[a][0] != start_node) or (LINKS_copy[j][1] != start_node and LINKS_copy[a][0] == end_node) or (LINKS_copy[j][1] != end_node and LINKS_copy[a][0] == start_node)):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                            elif new_weight + 2 * min_weight <= weight_of_path and (LINKS_copy[j][1] != start_node and LINKS_copy[a][0] != end_node) and (LINKS_copy[j][1] != end_node and LINKS_copy[a][0] != start_node):
                                                new_edge = [LINKS_copy[j][1], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(rev_list(LINKS_copy[j][3]))
                                                new_list.append(LINKS_copy[j][0])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                        new_weight -= LINKS_copy[a][2]
                            elif LINKS_copy[j][1] == i+1:
                                new_weight = LINKS_copy[j][2]
                                numbers_del.append(j)
                                for a in range(j+1, len_1):
                                    if ((LINKS_copy[a][0] == i+1 and LINKS_copy[a][1] != LINKS_copy[j][0]) or (LINKS_copy[a][1] == i+1 and LINKS_copy[a][0] != LINKS_copy[j][0])) and (set(LINKS_copy[a][3]).intersection(set(LINKS_copy[j][3])) == set() or (len(LINKS_copy[a][3]) == 0 and len(LINKS_copy[j][3]) == 0)):
                                        new_weight += LINKS_copy[a][2]
                                        if LINKS_copy[a][0] == i+1 and (new_weight == weight_of_path or new_weight + min_weight <= weight_of_path or new_weight + 2 * min_weight <= weight_of_path):
                                            if new_weight == weight_of_path and ((LINKS_copy[j][0] == start_node and LINKS_copy[a][1] == end_node) or (LINKS_copy[j][0] == end_node and LINKS_copy[a][1] == start_node)):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][1], new_weight] 
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                                check = False
                                                break
                                            elif new_weight + min_weight <= weight_of_path and ((LINKS_copy[j][0] == start_node and LINKS_copy[a][1] != end_node) or (LINKS_copy[j][0] == end_node and LINKS_copy[a][1] != start_node) or (LINKS_copy[j][0] != start_node and LINKS_copy[a][1] == end_node) or (LINKS_copy[j][0] != end_node and LINKS_copy[a][1] == start_node)):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][1], new_weight]
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                            elif new_weight + 2 * min_weight <= weight_of_path and (LINKS_copy[j][0] != start_node and LINKS_copy[a][1] != end_node) and (LINKS_copy[j][0] != end_node and LINKS_copy[a][1] != start_node):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][1], new_weight]
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(LINKS_copy[a][3])
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                        elif LINKS_copy[a][1] == i+1 and (new_weight == weight_of_path or new_weight + min_weight <= weight_of_path or new_weight + 2 * min_weight <= weight_of_path):
                                            if new_weight == weight_of_path and ((LINKS_copy[j][0] == start_node and LINKS_copy[a][0] == end_node) or (LINKS_copy[j][0] == end_node and LINKS_copy[a][0] == start_node)):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                                check = False
                                                break
                                            elif new_weight + min_weight <= weight_of_path and ((LINKS_copy[j][0] == start_node and LINKS_copy[a][0] != end_node) or (LINKS_copy[j][0] == end_node and LINKS_copy[a][0] != start_node) or (LINKS_copy[j][0] != start_node and LINKS_copy[a][0] == end_node) or (LINKS_copy[j][0] != end_node and LINKS_copy[a][0] == start_node)):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                            elif new_weight + 2 * min_weight <= weight_of_path and (LINKS_copy[j][0] != start_node and LINKS_copy[a][0] != end_node) and (LINKS_copy[j][0] != end_node and LINKS_copy[a][0] != start_node):
                                                new_edge = [LINKS_copy[j][0], LINKS_copy[a][0], new_weight]
                                                new_list = []
                                                new_list = new_list + list(LINKS_copy[j][3])
                                                new_list.append(LINKS_copy[j][1])
                                                new_list = new_list + list(rev_list(LINKS_copy[a][3]))
                                                new_edge.append(new_list)
                                                LINKS_copy.append(new_edge)
                                        new_weight -= LINKS_copy[a][2]
                            elif check == False or len(numbers_del) == del_deg:
                                break
                        if check == False:
                            break
                        else:
                            k = 0
                            for j in numbers_del:
                                LINKS_copy.pop(j - k)
                                k += 1
                            if len(LINKS_copy) == 1:
                                check = False

                    if check == False:
                        break
                    else:
                        deg_nodes = []
                        for j in range(len(LINKS)):
                            deg_nodes.append(0)
                        for j in range(len(LINKS_copy)):
                            deg_nodes[LINKS_copy[j][0]-1] += 1
                            deg_nodes[LINKS_copy[j][1]-1] += 1

                        del_deg = 10000000

                        for j in range(len(deg_nodes)):
                            if j+1 != start_node and j+1 != end_node and deg_nodes[j] < del_deg and deg_nodes[j] != 0:
                                del_deg = deg_nodes[j]

                        adjacency_list = {}
                        for j in STATIONS_copy:
                            adjacency_list[j] = set()

                        for j in LINKS_copy:
                            adjacency_list[j[0]].add(j[1])
                            adjacency_list[j[1]].add(j[0])

                elif check == False or deg_nodes.count(del_deg) == 0:
                    break

            if check == False:
                break

if LINKS_copy[-1][3] != list():
    for i in range(len(LINKS_copy)-1, -1, -1):
        if ((LINKS_copy[i][0] == start_node and LINKS_copy[i][1] == end_node) or (LINKS_copy[i][0] == end_node and LINKS_copy[i][1] == start_node)) and (LINKS_copy[i][2] == weight_of_path) and (LINKS_copy[i][3] != list()):
            check_weight = LINKS_copy[i][2]
            if LINKS_copy[i][0] == start_node:
                path = [start_node]
                path = path + list(LINKS_copy[i][3])
                path.append(end_node)
            elif LINKS_copy[i][0] == end_node:
                path = [start_node]
                path = path + list(rev_list(LINKS_copy[i][3]))
                path.append(end_node)
            break
else:
    check_weight = LINKS_copy[-1][2]
    if LINKS_copy[-1][0] == start_node:
        path = [start_node]
        path = path + list(LINKS_copy[-1][3])
        path.append(end_node)
    else:
        path = [start_node]
        path = path + list(rev_list(LINKS_copy[-1][3]))
        path.append(end_node)

for i in range(len(path)):
    path[i] -= 1

print('path', path)
print('weight', check_weight)

end_time = time.time()
print('Мой алгоритм', end_time - start_time)

30 118
path [4, 16, 11, 7, 0, 14, 5, 2, 21, 15]
weight of path 146

path [4, 14, 5, 2, 20, 8, 19, 23, 15]
weight path 146
count paths 52

Алгоритм Йена 0.0353083610534668

path [4, 14, 19, 12, 25, 15]
weight 146
Мой алгоритм 0.025033950805664062
